In [0]:
!pip install OpenNMT-py

In [0]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git
!cd OpenNMT-py && python setup.py install

In [2]:
!unzip Corpora.zip

Archive:  Corpora.zip
   creating: Corpora/
   creating: Corpora/En2Fa-Translation/
   creating: Corpora/En2Fa-Translation/Dev/
  inflating: Corpora/En2Fa-Translation/Dev/dev.en  
  inflating: Corpora/En2Fa-Translation/Dev/dev.fa  
   creating: Corpora/En2Fa-Translation/Test/
  inflating: Corpora/En2Fa-Translation/Test/test.en  
  inflating: Corpora/En2Fa-Translation/Test/test.fa0  
  inflating: Corpora/En2Fa-Translation/Test/test.fa1  
  inflating: Corpora/En2Fa-Translation/Test/test.fa2  
  inflating: Corpora/En2Fa-Translation/Test/test.fa3  
   creating: Corpora/En2Fa-Translation/Train/
  inflating: Corpora/En2Fa-Translation/Train/train.en  
  inflating: Corpora/En2Fa-Translation/Train/train.fa  
   creating: Corpora/Transliteration/
  inflating: Corpora/Transliteration/dev.en  
  inflating: Corpora/Transliteration/dev.fa  
  inflating: Corpora/Transliteration/test.en  
  inflating: Corpora/Transliteration/test.fa  
  inflating: Corpora/Transliteration/train.en  
  inflating: Corpor

In [0]:
def charactrize(path) :
  t = open(path).read()
  t = t.replace(' ','<b>')
  t = ' '.join(t)
  t = t.replace('< b >','<b>')
  open(path+'.ch','w').write(t)

for f in ['dev','test','train'] :
  for ff in ['fa','en'] :
    charactrize('Corpora/Transliteration/{}.{}'.format(f,ff))

In [5]:
!python OpenNMT-py/tools/learn_bpe.py -i Corpora/Transliteration/train.fa.ch -o Corpora/Transliteration/train.code.fa -s 10000

no pair has frequency >= 2. Stopping


In [6]:
!python OpenNMT-py/tools/learn_bpe.py -i Corpora/Transliteration/train.en.ch -o Corpora/Transliteration/train.code.en -s 10000

no pair has frequency >= 2. Stopping


In [0]:
!python OpenNMT-py/tools/apply_bpe.py -c Corpora/Transliteration/train.code.fa -i Corpora/Transliteration/train.fa.ch -o Corpora/Transliteration/train.bpe.fa

In [0]:
!python OpenNMT-py/tools/apply_bpe.py -c Corpora/Transliteration/train.code.fa -i Corpora/Transliteration/dev.fa.ch -o Corpora/Transliteration/dev.bpe.fa

In [0]:
!python OpenNMT-py/tools/apply_bpe.py -c Corpora/Transliteration/train.code.fa -i Corpora/Transliteration/test.fa.ch -o Corpora/Transliteration/test.bpe.fa

In [0]:
!python OpenNMT-py/tools/apply_bpe.py -c Corpora/Transliteration/train.code.en -i Corpora/Transliteration/train.en.ch -o Corpora/Transliteration/train.bpe.en

In [0]:
!python OpenNMT-py/tools/apply_bpe.py -c Corpora/Transliteration/train.code.en -i Corpora/Transliteration/dev.en.ch -o Corpora/Transliteration/dev.bpe.en

In [12]:
!onmt_preprocess -train_src Corpora/Transliteration/train.bpe.fa -train_tgt Corpora/Transliteration/train.bpe.en -valid_src Corpora/Transliteration/dev.bpe.fa -valid_tgt Corpora/Transliteration/dev.bpe.en -save_data ./demo

[2020-06-02 17:23:36,995 INFO] Extracting features...
[2020-06-02 17:23:36,996 INFO]  * number of source features: 0.
[2020-06-02 17:23:36,996 INFO]  * number of target features: 0.
[2020-06-02 17:23:36,996 INFO] Building `Fields` object...
[2020-06-02 17:23:36,996 INFO] Building & saving training data...
[2020-06-02 17:23:37,043 INFO] Building shard 0.
[2020-06-02 17:23:37,562 INFO]  * saving 0th train data shard to ./demo.train.0.pt.
[2020-06-02 17:23:37,908 INFO]  * tgt vocab size: 34.
[2020-06-02 17:23:37,908 INFO]  * src vocab size: 42.
[2020-06-02 17:23:37,943 INFO] Building & saving validation data...
[2020-06-02 17:23:37,963 INFO] Building shard 0.
[2020-06-02 17:23:37,998 INFO]  * saving 0th valid data shard to ./demo.valid.0.pt.


In [14]:
!onmt_train -data demo -save_model demo-model -train_steps 10000 -save_checkpoint_steps 1000 -valid_steps 1200 -world_size 1 -gpu_rank 0

[2020-06-02 17:24:57,120 INFO]  * src vocab size = 42
[2020-06-02 17:24:57,120 INFO]  * tgt vocab size = 34
[2020-06-02 17:24:57,120 INFO] Building model...
[2020-06-02 17:24:59,457 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(42, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(34, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (linear_out

In [15]:
!onmt_translate -model demo-model_step_10000.pt -src Corpora/Transliteration/test.fa.ch -output pred.txt -replace_unk -verbose

[2020-06-02 17:59:34,897 INFO] Translating shard 0.
/pytorch/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.

SENT 1: ['ن', 'ظ', 'ر', 'ت', '<b>', 'ب', 'ر', 'ا', 'م', '<b>', 'م', 'ه', 'م', 'ه']
PRED 1: n a z a r e t <b> b a r a a m <b> m o h e m m e
PRED SCORE: -0.0836

SENT 2: ['چ', 'و', 'ن', '<b>', 'د', 'و', 'س', 'ت', '<b>', 'ن', 'د', 'ا', 'ر', 'م', '<b>', 'د', 'ر', 'گ', 'ی', 'ر', '<b>', 'ا', 'ح', 'س', 'ا', 'س', 'ا', 'ت', '<b>', 'ب', 'ا', 'ش', 'ی', '<b>', 'و', '<b>', 'ت', 'ص', 'م', 'ی', 'م', '<b>', 'ب', 'گ', 'ی', 'ر', 'ی', '<b>', 'ن', 'م', 'ی', '\u200c', 'د', 'و', 'ن', 'م', '<b>', 'م', 'ن', 'ظ', 'و', 'ر', 'م', '<b>', 'ر', 'و', 'ش', 'ن', 'ه', '<b>', 'ی', 'ا', '<b>', 'ن', 'ه']
PRED 2: c h o n <b> d u s t <b> n a d a a r a m <b> d a r g i r <b> e <b> e h s a a s a a t e t <b> b a a s h i <b> o <b> t a 

In [16]:
!perl OpenNMT-py/tools/multi-bleu.perl Corpora/Transliteration/dev.en.ch < pred.txt

BLEU = 10.25, 72.2/39.3/15.3/5.5 (BP=0.463, ratio=0.565, hyp_len=59975, ref_len=106171)


In [0]:
1